In [5]:
import pandas as pd 
import numpy as np 
import collections 
from itertools import chain
from gensim.models import Word2Vec

In [6]:
train = pd.read_csv('./app/train.csv')
test = pd.read_csv('./app/test.csv')

train['name'] = train['name'].map(lambda x: x.split(' '))
test['name'] = test['name'].map(lambda x: x.split(' '))
train['description'] = train['description'].map(lambda x: x.split(' '))
test['description'] = test['description'].map(lambda x: x.split(' '))

train['l1'] = train['name'].map(lambda x: len(x))
test['l1'] = test['name'].map(lambda x: len(x))

train['l2'] = train['description'].map(lambda x: len(x))
test['l2'] = test['description'].map(lambda x: len(x))
train['len']= train.apply(lambda x: x['l1'] + x['l2'], axis=1 )
test['len']= test.apply(lambda x: x['l1'] + x['l2'], axis=1 )

In [7]:
train.describe(), test.describe()

(                id           l1           l2          len
 count  4199.000000  4199.000000  4199.000000  4199.000000
 mean   2099.000000     8.969278    37.087878    46.057156
 std    1212.291219     4.576621    79.204914    79.332999
 min       0.000000     2.000000     1.000000     4.000000
 25%    1049.500000     5.000000     6.000000    15.000000
 50%    2099.000000     8.000000    12.000000    22.000000
 75%    3148.500000    12.000000    26.000000    36.000000
 max    4198.000000    32.000000   946.000000   961.000000,
                id           l1           l2          len
 count  1800.00000  1800.000000  1800.000000  1800.000000
 mean    899.50000     8.942778    36.163889    45.106667
 std     519.75956     4.514390    77.122769    77.258960
 min       0.00000     2.000000     1.000000     3.000000
 25%     449.75000     5.000000     6.000000    15.000000
 50%     899.50000     8.000000    12.000000    21.000000
 75%    1349.25000    12.000000    27.000000    37.000000
 max

## 词向量

### char 

In [8]:
sentences = train['name'].values.tolist()+\
                test['name'].values.tolist() +\
                train['description'].values.tolist() +\
                test['description'].values.tolist()

char = collections.Counter(chain(*sentences))
print('total sentences {}, total char {}'.format(len(sentences), len(char)))

total sentences 11998, total char 2776


In [9]:
w2v = Word2Vec(sentences = sentences, vector_size=100, 
               window=5, min_count=1, workers=4, sg=1)
w2v.save('./checkpoint/char_min1_win5_sg_d100')

### Phrase 

In [10]:
from gensim.models.phrases import Phrases

In [23]:
phrase_model = Phrases(sentences, min_count=3, threshold=10)
new_sentences = phrase_model[sentences]

In [17]:
w2v = Word2Vec(sentences = new_sentences, vector_size=100, 
               window=5, min_count=1, workers=4, sg=1)
w2v.save('./checkpoint/phrase_min1_win5_sg_m5_d100')

In [18]:
phrase_model.save('./checkpoint/phrase_tokenizer')